In [8]:
%pip install prophet

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

def load_borg_trace_data(filepath):
    try:
        borg_df = pd.read_csv(filepath)
    except Exception as err:
        print("Error loading Borg trace file:", err)
        return None

    print("📊 Columns:", list(borg_df.columns))

    borg_df = borg_df.dropna(subset=['start_time', 'average_usage'])
    borg_df['start_time'] = pd.to_numeric(borg_df['start_time'], errors='coerce')
    borg_df = borg_df.dropna(subset=['start_time'])
    borg_df['start_time'] = pd.to_datetime(borg_df['start_time'], unit='ns', errors='coerce')
    borg_df = borg_df.dropna(subset=['start_time'])
    borg_df = borg_df.sort_values(by='start_time')
    borg_df.set_index('start_time', inplace=True)

    hourly_cpu_usage = borg_df['average_usage'].resample('h').mean().reset_index()

    print("✅ Preview of processed hourly CPU usage:")
    print(hourly_cpu_usage.head())

    return hourly_cpu_usage



In [2]:
from prophet import Prophet
import matplotlib.pyplot as plt

def forecast_cpu_usage(filepath, periods=24):
    df = load_borg_trace_data(filepath)

    if df is None or df.empty:
        print("No data available for forecasting.")
        return None

    df = df.rename(columns={'start_time': 'ds', 'average_usage': 'y'})

    model = Prophet()
    model.fit(df)

    future = model.make_future_dataframe(periods=periods, freq='h')
    forecast = model.predict(future)

    model.plot(forecast)
    plt.title("Forecasted CPU Usage for Next 24 Hours")
    plt.xlabel("Time")
    plt.ylabel("CPU Usage")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return forecast


In [11]:
def decide_resource_scaling(cpu_forecast_df, high_thresh=0.7, low_thresh=0.3):
    scaling_plan = []

    for _, row in cpu_forecast_df.iterrows():
        predicted_cpu = row['yhat']
        timestamp = row['ds']

        if predicted_cpu > high_thresh:
            action = "Scale UP"
        elif predicted_cpu < low_thresh:
            action = "Scale DOWN"
        else:
            action = "Maintain"

        scaling_plan.append((timestamp, action))

    return scaling_plan

def show_scaling_plan(plan):
    print("\n🧠 Scaling Recommendations (Next 24 Hours):")
    for ts, decision in plan:
        print(f"{ts.strftime('%Y-%m-%d %H:%M')} → {decision}")


In [12]:
def execute_scaling_plan(plan):
    print("\n⚙️ Simulating Scaling Execution:")
    for ts, action in plan:
        print(f"🛠️  [{ts.strftime('%Y-%m-%d %H:%M')}] → Action: {action}")


In [ ]:
print("🚀 Smart Auto-Scaler System Starting...")

file_path = r"C:\Users\dell\OneDrive\Desktop\smart_autoscaler\borg_traces_data.csv"

data = load_borg_trace_data(file_path)

if data is not None:
    forecast = forecast_cpu_usage(file_path, periods=24)

    if forecast is not None:
        print("\n📈 Forecast sample:")
        print(forecast[['ds', 'yhat']].tail())

        scaling_plan = decide_resource_scaling(forecast)
        show_scaling_plan(scaling_plan)
        execute_scaling_plan(scaling_plan)
    else:
        print("❌ Forecasting failed.")
else:
    print("❌ Failed to load data.")


🚀 Smart Auto-Scaler System Starting...
📊 Columns: ['Unnamed: 0', 'time', 'instance_events_type', 'collection_id', 'scheduling_class', 'collection_type', 'priority', 'alloc_collection_id', 'instance_index', 'machine_id', 'resource_request', 'constraint', 'collections_events_type', 'user', 'collection_name', 'collection_logical_name', 'start_after_collection_ids', 'vertical_scaling', 'scheduler', 'start_time', 'end_time', 'average_usage', 'maximum_usage', 'random_sample_usage', 'assigned_memory', 'page_cache_memory', 'cycles_per_instruction', 'memory_accesses_per_instruction', 'sample_rate', 'cpu_usage_distribution', 'tail_cpu_usage_distribution', 'cluster', 'event', 'failed']
